In [1]:
import pandas as pd
from xgboost import XGBRegressor
from sklearn.ensemble import RandomForestRegressor

In [2]:
stats = pd.read_csv("player_mvp_stats.csv")
del stats["Unnamed: 0"]
stats_all_players = stats.fillna(0)
mvp_players = stats_all_players[(stats_all_players["FG%"] > 0.384) & (stats_all_players["MP"] > 25.6) & (stats_all_players["FT%"] > 0.422) & (stats_all_players["PTS"] > 4.7) & (stats_all_players["VORP"] > 1.1)& (stats_all_players["W/L%"] > 0.22)]
mvp_players

,Player,Pos,Age,Tm,G,GS,MP,FG,FGA,FG%,...,Pts Max,Share,Team,W,L,W/L%,GB,PS/G,PA/G,SRS
1,Byron Scott,SG,29.0,LAL,82.0,82.0,32.1,6.1,12.8,0.477,...,0.0,0.000,Los Angeles Lakers,58.0,24.0,0.707,5.0,106.3,99.6,6.73
4,James Worthy,SF,29.0,LAL,78.0,74.0,38.6,9.2,18.7,0.492,...,0.0,0.000,Los Angeles Lakers,58.0,24.0,0.707,5.0,106.3,99.6,6.73
6,Magic Johnson,PG,31.0,LAL,79.0,79.0,37.1,5.9,12.4,0.477,...,960.0,0.518,Los Angeles Lakers,58.0,24.0,0.707,5.0,106.3,99.6,6.73
8,Sam Perkins,PF,29.0,LAL,73.0,66.0,34.3,5.0,10.2,0.495,...,0.0,0.000,Los Angeles Lakers,58.0,24.0,0.707,5.0,106.3,99.6,6.73
11,Vlade Divac,C,22.0,LAL,82.0,81.0,28.2,4.4,7.8,0.565,...,0.0,0.000,Los Angeles Lakers,58.0,24.0,0.707,5.0,106.3,99.6,6.73
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
15211,Kendall Gill,SF,31.0,NJN,76.0,75.0,31.0,5.2,12.6,0.414,...,0.0,0.000,New Jersey Nets,31.0,51.0,0.378,21.0,98.0,99.0,-1.18
15212,Kerry Kittles,SG,25.0,NJN,62.0,61.0,30.6,4.9,11.3,0.437,...,0.0,0.000,New Jersey Nets,31.0,51.0,0.378,21.0,98.0,99.0,-1.18
15218,Stephon Marbury,PG,22.0,NJN,74.0,74.0,38.9,7.7,17.8,0.432,...,0.0,0.000,New Jersey Nets,31.0,51.0,0.378,21.0,98.0,99.0,-1.18
15220,Giannis Antetokounmpo,SF,22.0,MIL,80.0,80.0,35.6,8.2,15.7,0.521,...,1010.0,0.007,Milwaukee Bucks,42.0,40.0,0.512,9.0,103.6,103.8,-0.45


In [3]:
stats_2024 = pd.read_csv("2024_stats.csv")
del stats_2024["Unnamed: 0"]

In [4]:
stats_2024 = stats_2024.fillna(0)
stats_2024 = stats_2024[(stats_2024["FG%"] > 0.384) & (stats_2024["MP"] > 25.6) & (stats_2024["FT%"] > 0.422) & (stats_2024["PTS"] > 4.7) & (stats_2024["VORP"] > 1.1)& (stats_2024["W/L%"] > 0.22)]
condition = (stats_2024['W'] + stats_2024['L']) - stats_2024['G'] < 17
stats_2024 = stats_2024[condition]

In [5]:
# pd.set_option('display.max_rows', None)
# len(stats_2024["Player"])

In [28]:
predictors = ['Age', 'G', 'GS', 'MP', 'FG', 'FGA', 'FG%', '3P',
       '3PA', '3P%', '2P', '2PA', '2P%', 'eFG%', 'FT', 'FTA', 'FT%', 'ORB',
       'DRB', 'TRB', 'AST', 'STL', 'BLK', 'TOV', 'PF', 'PTS', 'Year', 'W', 'L', 'W/L%', 'GB', 'PS/G',
       'PA/G', 'SRS','PER','TS%', '3PAr', 'FTr', 'ORB%', 'DRB%', 'TRB%', 'AST%', 'STL%', 'BLK%',
       'TOV%', 'USG%', 'OWS', 'DWS', 'WS', 'WS/48', 'OBPM', 'DBPM', 'BPM',
       'VORP']

#Predicting on 2024 players stats. Training model on data from previous years (1990 - 2023)
train = mvp_players
test = stats_2024

X_train = train[predictors]
y_train = train["Share"]

X_test = test[predictors]
# y_test = test["Share"]

model = XGBRegressor(learning_rate = 0.05, max_depth = 7, subsample = 0.5)

model.fit(X_train, y_train)
predicted_winners = model.predict(X_test)

predicted_winners = pd.DataFrame(predicted_winners, columns=["Predictions"], index = test.index)
combinations_XGB = pd.concat([test, predicted_winners], axis = 1)
combinations_XGB = combinations_XGB.sort_values("Predictions", ascending = False)
# combinations = combinations.sort_values("Share", ascending = False)
# combinations["Rank"] = list(range(1, combinations.shape[0] + 1))
# combinations = combinations.sort_values("Predictions", ascending = False)
# combinations["Predicted_Rank"] = list(range(1, combinations.shape[0] + 1))

In [29]:
combinations_XGB.head(5)

,Player,Pos,Age,Tm,G,GS,MP,FG,FGA,FG%,...,VORP,Team,W,L,W/L%,GB,PS/G,PA/G,SRS,Predictions
67,Nikola Jokić,C,28,DEN,72,72,34.5,10.3,17.8,0.579,...,9.7,Denver Nuggets,52,23,0.693,0.5,114.6,109.8,4.91,0.982266
122,Shai Gilgeous-Alexander,PG,25,OKC,71,71,34.4,10.7,19.9,0.539,...,6.8,Oklahoma City Thunder,52,22,0.703,0.0,120.4,113.0,7.22,0.760013
7,Giannis Antetokounmpo,PF,29,MIL,69,69,35.2,11.5,18.8,0.614,...,6.8,Milwaukee Bucks,47,27,0.635,0.0,120.4,116.8,3.25,0.679755
30,Luka Dončić,PG,24,DAL,65,65,37.4,11.6,23.6,0.489,...,7.4,Dallas Mavericks,45,29,0.608,0.0,118.7,116.3,2.58,0.539351
165,Jayson Tatum,PF,25,BOS,69,69,35.9,9.2,19.5,0.474,...,4.5,Boston Celtics,58,16,0.784,0.0,120.9,109.5,11.05,0.093323


In [35]:
combinations_XGB.head(5).to_csv("XGB_predictions.csv")

In [36]:
model = RandomForestRegressor(bootstrap=True, max_depth=40, max_features = 'sqrt', min_samples_leaf=2, min_samples_split=2, n_estimators = 200)

model.fit(X_train, y_train)
predicted_winners = model.predict(X_test)

predicted_winners = pd.DataFrame(predicted_winners, columns=["Predictions"], index = test.index)
combinations_RF = pd.concat([test, predicted_winners], axis = 1)
combinations_RF = combinations_RF.sort_values("Predictions", ascending = False)

In [37]:
combinations_RF.head(5)

,Player,Pos,Age,Tm,G,GS,MP,FG,FGA,FG%,...,VORP,Team,W,L,W/L%,GB,PS/G,PA/G,SRS,Predictions
67,Nikola Jokić,C,28,DEN,72,72,34.5,10.3,17.8,0.579,...,9.7,Denver Nuggets,52,23,0.693,0.5,114.6,109.8,4.91,0.773491
122,Shai Gilgeous-Alexander,PG,25,OKC,71,71,34.4,10.7,19.9,0.539,...,6.8,Oklahoma City Thunder,52,22,0.703,0.0,120.4,113.0,7.22,0.595937
7,Giannis Antetokounmpo,PF,29,MIL,69,69,35.2,11.5,18.8,0.614,...,6.8,Milwaukee Bucks,47,27,0.635,0.0,120.4,116.8,3.25,0.515567
30,Luka Dončić,PG,24,DAL,65,65,37.4,11.6,23.6,0.489,...,7.4,Dallas Mavericks,45,29,0.608,0.0,118.7,116.3,2.58,0.376350
165,Jayson Tatum,PF,25,BOS,69,69,35.9,9.2,19.5,0.474,...,4.5,Boston Celtics,58,16,0.784,0.0,120.9,109.5,11.05,0.146669


In [38]:
combinations_RF.head(5).to_csv("RF_predictions.csv")